In [16]:
import xml.etree.ElementTree as ET
import MeCab

tree = ET.parse('../data3/琉球大学オントロジー.xml')
root = tree.getroot() #タグと属性の情報を持つ

In [17]:
label_role={}
def down_node(node, indent=0, label = 0):
    #print("{}{} {}".format('    ' * indent, node.tag, node.attrib))
    for child in node:
        if child.tag == 'LABEL':
            label = child.text
            print("LABEL:{}".format(child.text))
        if child.tag == 'SLOT':
            print("{}Role:{}  num:{}  value:{} constraint{}".format('    ', child.attrib["role"], child.attrib["num"], child.attrib["value"], child.attrib["class_constraint"]))
            role = {'role': child.attrib["role"], 'num': child.attrib["num"], 'value': child.attrib["value"], 'constraint': child.attrib["class_constraint"]}
            label_role.setdefault(label, []).append(role)
        down_node(child, indent + 1, label)

In [18]:
down_node(root)

LABEL:琉球大学
LABEL:大学院
    Role:年数  num:1  value:2 constraintAny
LABEL:学費
    Role:授業料  num:  value: constraint年間
    Role:入学料  num:1  value: constraintAny
    Role:検定料  num:1  value: constraintAny
LABEL:年間
    Role:前期  num:1  value: constraintAny
    Role:後期  num:1  value: constraintAny
LABEL:Any
LABEL:学部
    Role:年数  num:1  value:4 constraintAny


In [26]:
search_queri = '琉球大学 4年間 学費'
search_sentence = '琉球大学の4年間の学費はいくらか'
search_school_fees = [{"授業料":{"年額":535800,"前期": 267900, "後期": 267900}},{"入学料":282000},{"検定料":17000}]

In [27]:
search_sentence

'琉球大学の4年間の学費はいくらか'

In [28]:
for key in label_role:
    for index,label in enumerate(label_role[key]):
        for fees in search_school_fees:
            fee = [fee for fee in fees]
            if label['role'] in fee:
                label['value'] = fees[fee[0]]
print(label_role)

{'大学院': [{'role': '年数', 'num': '1', 'value': '2', 'constraint': 'Any'}], '学費': [{'role': '授業料', 'num': '', 'value': {'年額': 535800, '前期': 267900, '後期': 267900}, 'constraint': '年間'}, {'role': '入学料', 'num': '1', 'value': 282000, 'constraint': 'Any'}, {'role': '検定料', 'num': '1', 'value': 17000, 'constraint': 'Any'}], '年間': [{'role': '前期', 'num': '1', 'value': '', 'constraint': 'Any'}, {'role': '後期', 'num': '1', 'value': '', 'constraint': 'Any'}], '学部': [{'role': '年数', 'num': '1', 'value': '4', 'constraint': 'Any'}]}


In [35]:
import re
mecab = MeCab.Tagger()
search_mecab = mecab.parse(search_sentence)
lines = search_mecab.split('\n')
items = (re.split('[\t,]', line) for line in lines)
num_dic = {}
for count,item in enumerate(items):
    if item[0] not in ('EOS', '', 't', 'ー') and item[2] == '数':
        num_dic[next(items)[0]] = item[0]
num_dic

{'年間': '4'}

In [30]:
label_role

{'大学院': [{'role': '年数', 'num': '1', 'value': '2', 'constraint': 'Any'}],
 '学費': [{'role': '授業料',
   'num': '',
   'value': {'年額': 535800, '前期': 267900, '後期': 267900},
   'constraint': '年間'},
  {'role': '入学料', 'num': '1', 'value': 282000, 'constraint': 'Any'},
  {'role': '検定料', 'num': '1', 'value': 17000, 'constraint': 'Any'}],
 '年間': [{'role': '前期', 'num': '1', 'value': '', 'constraint': 'Any'},
  {'role': '後期', 'num': '1', 'value': '', 'constraint': 'Any'}],
 '学部': [{'role': '年数', 'num': '1', 'value': '4', 'constraint': 'Any'}]}

In [31]:
wakati = MeCab.Tagger("-Owakati")
search_wakati = wakati.parse(search_sentence) #琉球大学 の 4 年間 の 授業 料 は いくら か 
queris = search_wakati.split(" ")
queris

['琉球大学', 'の', '4', '年間', 'の', '学費', 'は', 'いくら', 'か', '\n']

In [32]:
answer = 0
ontrogy_queris = []
dic = {}
nonum = 0
for queri in queris:
    ontrogy_queri = queri
    if queri in label_role: # オントロジー要素の中にクエリがあるか？
        role = []
        for label in label_role[queri]:
            if label['value'] and label['num']:
                answer += int(label['value']) * int(label['num'])
            elif label['value'] and not label['num']: #変数的に値が増加する数値情報を持つ
                if label['constraint'] in label_role: #変化する参照の値があるなら
                    for cons in label_role[label['constraint']]:
                        nonum += label['value'][cons['role']]
                if label['constraint'] in num_dic:
                    answer += int(nonum) * int(num_dic[label['constraint']])
            #else:
            #    role.append(label['role'])
        #dic[queri] = role
        #ontrogy_queris.append(dic)
    #else:
        #ontrogy_queris.append(ontrogy_queri)
print(answer)

2442200


In [33]:
search_sentence

'琉球大学の4年間の学費はいくらか'

In [34]:
answer

2442200